# TP2 - Julien Vanbelle

## Imports

In [1]:
import os
import yake
from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences
year = 1890

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julienvanbelle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Création d'une liste de mots à ignorer
ignored = set(["conseil communal", "conseil général", "conseil d'administration","conseil supérieur","conseil provincial"])
ignored

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Lister les Fichiers
data_path = "/Users/julienvanbelle/Documents/GitHub/tac/data/txt"
files = os.listdir(data_path)

In [ ]:
# Imprimer le nombre de fichiers identifiés
len(files)

In [ ]:
# Enlever les fichiers qui ne commencent pas par Bxl_
date_files = [f for f in files if f.startswith('Bxl_1890')]
len(date_files)

In [ ]:
for f in sorted(date_files):
    text = open(os.path.join(data_path, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower())  ##.lower() ==> permet de mettre le texte en miniscule
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville"]
sw = set(sw)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in date_files:
    with open(os.path.join(data_path, txt), 'r') as f:
        content_list.append(f.read())

len(content_list)

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '/Users/julienvanbelle/Documents/GitHub/tac/data'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r') as f:
    before = f.read()

before[:500]

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r') as f:
    after = f.read()

after[:500]

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

In [2]:
nlp = spacy.load('fr_core_news_md')

In [3]:
# Charger le texte
n=900000
text = open("/Users/julienvanbelle/Documents/GitHub/tac/data/all.txt", encoding='utf-8').read()[:n]

In [4]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: user 2min 2s, sys: 8.03 s, total: 2min 10s
Wall time: 2min 18s


In [40]:
# Compter les entités
people = defaultdict(int)
organisation = defaultdict(int)
places = defaultdict(int)

for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    if ent.label_ == "ORG" and len(ent.text) > 3:
        organisation[ent.text] += 1
    if ent.label_ == "GPE" and len(ent.text) > 3:
        places[ent.text] += 1
      
    
      

In [41]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
sorted_organisation = sorted(organisation.items(), key=lambda kv: kv[1], reverse=True)
sorted_places = sorted(places.items(), key=lambda kv: kv[1], reverse=True)


for person, freq in sorted_people:
    print(f"{person} (PER)apparait {freq} fois dans le corpus")

for organisation, freq in sorted_organisation:
    print(f"{organisation} (ORG)apparait {freq} fois dans le corpus")

for places, freq in sorted_places:
    print(f"{places} (GPE)apparait {freq} fois dans le corpus")

Gendebien (PER)apparait 118 fois dans le corpus
q u i (PER)apparait 101 fois dans le corpus
Bortier (PER)apparait 75 fois dans le corpus
Bourgmestre (PER)apparait 56 fois dans le corpus
Doucet (PER)apparait 44 fois dans le corpus
Messieurs (PER)apparait 34 fois dans le corpus
Vanderlinden (PER)apparait 31 fois dans le corpus
Anspach (PER)apparait 30 fois dans le corpus
Doncker (PER)apparait 29 fois dans le corpus
M. Bortier (PER)apparait 25 fois dans le corpus
Page (PER)apparait 25 fois dans le corpus
Cans (PER)apparait 24 fois dans le corpus
Fontainas (PER)apparait 23 fois dans le corpus
q u ' i (PER)apparait 18 fois dans le corpus
échevin Orts (PER)apparait 17 fois dans le corpus
j o u r (PER)apparait 15 fois dans le corpus
Cluysenaar (PER)apparait 15 fois dans le corpus
Schumacher (PER)apparait 15 fois dans le corpus
é t a i (PER)apparait 10 fois dans le corpus
M. Gendebien (PER)apparait 10 fois dans le corpus
M. Cluysenaar (PER)apparait 10 fois dans le corpus
M. Blaes (PER)apparait